###### Preprocessing News Data

In [1]:
# import libraries
import numpy as np
import pandas as pd  
import csv, re
import nltk

from langdetect import detect
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


In [28]:
# setting some options of pandas

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 50)

In [29]:
df = pd.read_csv("D:/BE_Project/Data_json_csv/csv_data_news_temp.csv", encoding = "latin1")
# df.head()
# df.drop([df.iloc[:, 705:712]], axis=1, inplace = True)
# df.to_csv("ouput_news_data.csv")

In [4]:

columns = df.columns
size = df.size
shape = df.shape

print("\n"+"size(rows*columns) - "+str(size)+"\n")
print("columns - "+str(columns)+"\n")
print("shape - "+str(shape)+"\n")



size(rows*columns) - 39011830

columns - Index(['Sr', 'author', 'title', 'timestamp', 'text', 'spam_score',
       'performance_score', 'domain_rank', 'facebook_likes',
       'facebook_comments',
       ...
       'Unnamed: 705', 'Unnamed: 706', 'Unnamed: 707', 'Unnamed: 708',
       'Unnamed: 709', 'Unnamed: 710', 'Unnamed: 711', 'Unnamed: 712',
       'Unnamed: 713', 'Unnamed: 714'],
      dtype='object', length=715)

shape - (54562, 715)



In [30]:
df.head()

,Sr,author,title,timestamp,text,spam_score,performance_score,domain_rank,facebook_likes,facebook_comments,...,Unnamed: 705,Unnamed: 706,Unnamed: 707,Unnamed: 708,Unnamed: 709,Unnamed: 710,Unnamed: 711,Unnamed: 712,Unnamed: 713,Unnamed: 714
0,1,NaN,Why Bitcoin and Crypto Have No Future â Think Outside the Blox â Medium,2018-10-19T03:00:00.000+03:00,"Cryptoasset enthusiast and investor. economics. @outsidethebloxx Oct 18 Why Bitcoin and Crypto Have No Future Why do Bitcoin and cryptoassets have no futureâ¦ when the NYSE is creating the cryptocurrency trading platform Bakkt in collaboration with Microsoft and Starbucks with physically backed Bitcoin futures contracts. when Fidelity is offering it to its clients and has been mining it since 2015. when Steve Wozniak is co-founding a cryptoasset investment firm. when Katy Perry IGâd this! Cryptos coming at you like a dark horse when Brendan Eich , creator of JavaScript and Mozilla, started Brave Browser with a built-in ad-blocker and Basic Attention Token (BAT) to reward content creators. when Amazon Web Services partnered with QTUM. when the Bill & Melinda Gates Foundation is utilizing Rippleâs interledger protocol to help with payment services for the financially impoverished and unbanked. when IBM is partnering with Stellar Lumens (XLM) for cross-border payment solutions. w...",0,7,380,1528,519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,"Deduct tithes straight from workersâ salaries, Archbishop begs govt â Punch Newspapers",2018-10-29T02:00:00.000+02:00,"Deduct tithes straight from workersâ salaries, Archbishop begs govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV.\nThe Archbishop of Kampala, Cyprian Kizito Lwanga, has suggested that the government of Uganda deduct tithe from workersâ salaries.\nSpeaking during a mass, the Catholic Archbishop said many Christians donât tithe and this affects church projects, reports Citizen Kenya .\nâWhenever we ask for tithe, everyone gives only what they have at that time. But the Bible says a tenth of whatever you earn belongs to the church,â said Archbishop Lwanga.\nâGive me your support as I front this proposal because it is good for us. Arenât you tired of putting money in the baskets all the time?â\nREAD ALSO: BREAKING: Governors summon emergency meeting over minimum wage\nArchbishop Lwanga said he wanted Uganda to go the German route, where Germans, who are registered as Catholics, Protestants or Jews have a church tax ( Kirchensteu...",0,10,2692,4005,4914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Mark Townsend,Tommy Robinson âstands to make Â£1mâ on US speaking tour | UK news | The Guardian,2018-10-27T18:56:00.000+03:00,"Tommy Robinson could make more than Â£1m from a potential trip to the US next month, making him one of the best funded far-right figures since the second world war, an analysis by anti-fascist campaigners says.\nThe English Defence League founder, whose real name is Stephen Yaxley-Lennon, has been invited to give an address in Washington by members of Congress and is waiting to see if US authorities will grant a visa for the event, scheduled for 14 November.\nThe Middle East Forum (MEF) , a US neo-conservative thinktank, and the rightwing David Horowitz Freedom Center invited the 35-year-old, who, despite repeatedly falling foul of the British justice system, has growing support in the US. Robinson is on bail after being charged with contempt of court, and has a string of previous convictions in Britain.\nJoe Mulhall, a senior researcher at Hope not Hate , said: âThe inevitable media coverage that will surround the visit will no doubt lead to an increase in his already significan...",0,10,170,8429,8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Kate Rooney,Fidelity launches trade execution and custody for cryptocurrencies,2018-10-15T20:30:00.000+03:00,"Bitcoin Fidelity just made it easier for hedge funds and other pros to in

#### Remove extra columns

In [6]:
df.drop(df.iloc[:, 11:715], axis=1, inplace = True)
df.head()
# df.to_csv("output.csv")

,Sr,author,title,timestamp,text,spam_score,performance_score,domain_rank,facebook_likes,facebook_comments,facebook_shares
0,1,NaN,Why Bitcoin and Crypto Have No Future â Think Outside the Blox â Medium,2018-10-19T03:00:00.000+03:00,Cryptoasset enthusiast and investor. economics. @outsidethebloxx Oct 18 Why Bitcoin and Crypto Have No Future Why do Bitcoin and cryptoassets have no futureâ¦ when the NYSE is creating the cryptocurrency trading platform Bakkt in collaboration with Micros...,0,7,380,1528,519,775
1,2,NaN,"Deduct tithes straight from workersâ salaries, Archbishop begs govt â Punch Newspapers",2018-10-29T02:00:00.000+02:00,"Deduct tithes straight from workersâ salaries, Archbishop begs govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV.\nThe Archbishop of Kampala, Cyprian Kizito Lwanga, has suggested that the government of Uganda ded...",0,10,2692,4005,4914,1426
2,3,Mark Townsend,Tommy Robinson âstands to make Â£1mâ on US speaking tour | UK news | The Guardian,2018-10-27T18:56:00.000+03:00,"Tommy Robinson could make more than Â£1m from a potential trip to the US next month, making him one of the best funded far-right figures since the second world war, an analysis by anti-fascist campaigners says.\nThe English Defence League founder, whose re...",0,10,170,8429,8507,1617
3,4,Kate Rooney,Fidelity launches trade execution and custody for cryptocurrencies,2018-10-15T20:30:00.000+03:00,"Bitcoin Fidelity just made it easier for hedge funds and other pros to invest in cryptocurrencies Fidelity Investments, which administers more than $7.2 trillion in client assets, announced a new and separate company called Fidelity Digital Asset Services ...",0,7,767,1431,299,783
4,5,theblaze.com,WATCH: Hillary Clinton jokes about black Democrats: âI know they all look alikeâ | TheBlaze,2018-10-30T07:20:00.000+02:00,Hillary Clinton appeared quite amused with herself after making a joke during an interview that could be interpreted as racist against black people.\nClinton sat down with Kara Swisher for the Recode Decode podcast over the weekend. Swisher asked Clinton a...,0,10,1287,4289,1107,1445


#### Analyze Dimensions

In [7]:
columns = df.columns
size = df.size
shape = df.shape

print("\n"+"size(rows*columns) - "+str(size)+"\n")
print("columns - "+str(columns)+"\n")
print("shape - "+str(shape)+"\n")



size(rows*columns) - 600182

columns - Index(['Sr', 'author', 'title', 'timestamp', 'text', 'spam_score',
       'performance_score', 'domain_rank', 'facebook_likes',
       'facebook_comments', 'facebook_shares'],
      dtype='object')

shape - (54562, 11)



### Handling Missing Values

In [8]:
df.isnull().values.any()

True

In [9]:
df.isnull().sum()

Sr                       0
author                7268
title                 1789
timestamp             2053
text                  2304
spam_score            2672
performance_score     2905
domain_rank          46069
facebook_likes        3186
facebook_comments     3278
facebook_shares       3357
dtype: int64

In [10]:
# Drop domain rank column - 7200 missing values
df.drop("domain_rank", axis=1,inplace = True)


In [11]:
# Drop rows with missing values in text columns
df['text'].replace("", np.nan, inplace=True)
df.dropna(subset=['text'], inplace=True)

# Drop rows with missing values in timestamp column
df['timestamp'].replace("", np.nan, inplace=True)
df.dropna(subset=['timestamp'], inplace=True)


In [12]:
df

,Sr,author,title,timestamp,text,spam_score,performance_score,facebook_likes,facebook_comments,facebook_shares
0,1,NaN,Why Bitcoin and Crypto Have No Future â Think Outside the Blox â Medium,2018-10-19T03:00:00.000+03:00,Cryptoasset enthusiast and investor. economics. @outsidethebloxx Oct 18 Why Bitcoin and Crypto Have No Future Why do Bitcoin and cryptoassets have no futureâ¦ when the NYSE is creating the cryptocurrency trading platform Bakkt in collaboration with Micros...,0,7,1528,519,775
1,2,NaN,"Deduct tithes straight from workersâ salaries, Archbishop begs govt â Punch Newspapers",2018-10-29T02:00:00.000+02:00,"Deduct tithes straight from workersâ salaries, Archbishop begs govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV.\nThe Archbishop of Kampala, Cyprian Kizito Lwanga, has suggested that the government of Uganda ded...",0,10,4005,4914,1426
2,3,Mark Townsend,Tommy Robinson âstands to make Â£1mâ on US speaking tour | UK news | The Guardian,2018-10-27T18:56:00.000+03:00,"Tommy Robinson could make more than Â£1m from a potential trip to the US next month, making him one of the best funded far-right figures since the second world war, an analysis by anti-fascist campaigners says.\nThe English Defence League founder, whose re...",0,10,8429,8507,1617
3,4,Kate Rooney,Fidelity launches trade execution and custody for cryptocurrencies,2018-10-15T20:30:00.000+03:00,"Bitcoin Fidelity just made it easier for hedge funds and other pros to invest in cryptocurrencies Fidelity Investments, which administers more than $7.2 trillion in client assets, announced a new and separate company called Fidelity Digital Asset Services ...",0,7,1431,299,783
4,5,theblaze.com,WATCH: Hillary Clinton jokes about black Democrats: âI know they all look alikeâ | TheBlaze,2018-10-30T07:20:00.000+02:00,Hillary Clinton appeared quite amused with herself after making a joke during an interview that could be interpreted as racist against black people.\nClinton sat down with Kara Swisher for the Recode Decode podcast over the weekend. Swisher asked Clinton a...,0,10,4289,1107,1445
5,6,channelnewsasia.com,Live-streaming of child sex abuse spreads in the Philippines - Channel NewsAsia,2018-10-27T01:02:00.000+03:00,"\nMANILA: Victor Lorenzo has spent much of his life chasing down criminals and helping their victims. For a law enforcer with years of experience, much of what he does is now routine.\nYet there is one type of crime which the veteran chief of the Philippin...",0.82,10,5971,528,1085
6,7,Michael del Castillo,Fidelity Launches Institutional Platform For Bitcoin And Ethereum,2018-10-15T20:58:00.000+03:00,I cover enterprise adoption of blockchain and cryptocurrency. Fidelity Investments Reportedly Plans Layoffs 2017 â The Boston Globe\nFidelity Investments is spinning off a stand-alone company dedicated to bringing cryptocurrencies to institutional invest...,0,6,1462,313,677
7,8,"Rosalie Chan , Business Insider US","The 3rd most powerful supercomputer in the world was turned on at a classified government lab in California â Hereâs what the 7,000 square foot âmini cityâ of processing power is like up close",2018-10-27T17:30:00.000+03:00,"Rosalie Chan\nThe third most powerful supercomputer in the world has been completed and unveiled: the Sierra. This supercomputer, which can do 125 quadrillion calculations in a second, will be used to create simulations that can test how safe and reliable ...",0,7,3662,433,756
8,9,NaN,CRA phone scam: Arrests made as Indian call centres raided | CTV News,2018-11-07T02:00:00.000+02:00,"Last month, Guy-Paul Larocque received a call from a man claiming to be a representative of the U.S. Internal Revenue Service.\nThe man claimed Larocque had qualified to receive a grant and needed to share some of his personal financial information before ...",0,8,3606,775,832
9,10,NaN,"The 3rd most powerful supercomputer in the world was turned on at a classified government

In [13]:
df.isnull().sum()

Sr                      0
author               5961
title                  12
timestamp               0
text                    0
spam_score            373
performance_score     606
facebook_likes        875
facebook_comments     967
facebook_shares      1046
dtype: int64

### Handling Duplicate Values

In [14]:
print("no. of duplicates : "+str(df.duplicated().sum()))
# remove duplicated rows
df.drop_duplicates(keep="first", inplace=True)

no. of duplicates : 273


In [15]:
df.shape

(51977, 10)

In [16]:
# ignored for now
df.duplicated(subset=['text', 'author']).sum()

2704

### Handle URLs, Mentions, '\n', Non-ASCII characters

In [17]:
def preprocess(row):
#   mention
    row = re.sub(r'@[A-Za-z0-9]+',' ',row)
#   url
    row = re.sub('https?://[A-Za-z0-9./]+',' ',row)
    row = re.sub('[A-Za-z0-9].com', ' ', row)
#   \n
    row = re.sub('/n', ' ', row)    
#   row = re.sub("[^a-zA-Z]", "", row)
#   non-ascii characters
    row = re.sub(r'[^\x00-\x7F]+',' ',row)
    return row

df['preprocess'] = df['text'].apply(preprocess)

### Handle other languages

In [18]:
count = 0
# for feature not found
count2 = 0 
for i in range(1, len(df)):
    try: 
        if (detect(str(df.loc[i,'text'])) != "en"):
#             print(i)
            df.drop(i, axis = 0, inplace = True)
            count +=1
    except Exception:
#         print("feature not Found")
        count2 += 1
        
print("other language "+str(count))
print("feature not found for "+str(count2))



other language 314
feature not found for 2523


In [19]:
df.shape

(51663, 11)

### Remove Stopwords

In [20]:
# download this once
# import nltk
# nltk.download('stopwords')

stop = stopwords.words("english")
print(df['preprocess']+"\n")
df['preprocess'] =  df['preprocess'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df['preprocess'])

0        Cryptoasset enthusiast and investor. economics.  Oct 18 Why Bitcoin and Crypto Have No Future Why do Bitcoin and cryptoassets have no future when the NYSE is creating the cryptocurrency trading platform Bakkt in collaboration with Microsoft and Starbucks w...
1        Deduct tithes straight from workers salaries, Archbishop begs govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV.\nThe Archbishop of Kampala, Cyprian Kizito Lwanga, has suggested that the government of Uganda deduct...
2        Tommy Robinson could make more than 1m from a potential trip to the US next month, making him one of the best funded far-right figures since the second world war, an analysis by anti-fascist campaigners says.\nThe English Defence League founder, whose real...
3        Bitcoin Fidelity just made it easier for hedge funds and other pros to invest in cryptocurrencies Fidelity Investments, which administers more than $7.2 trillion in client assets, anno

0        Cryptoasset enthusiast investor. economics. Oct 18 Why Bitcoin Crypto Have No Future Why Bitcoin cryptoassets future NYSE creating cryptocurrency trading platform Bakkt collaboration Microsoft Starbucks physically backed Bitcoin futures contracts. Fidelity...
1        Deduct tithes straight workers salaries, Archbishop begs govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV. The Archbishop Kampala, Cyprian Kizito Lwanga, suggested government Uganda deduct tithe workers salaries. ...
2        Tommy Robinson could make 1m potential trip US next month, making one best funded far-right figures since second world war, analysis anti-fascist campaigners says. The English Defence League founder, whose real name Stephen Yaxley-Lennon, invited give addr...
3        Bitcoin Fidelity made easier hedge funds pros invest cryptocurrencies Fidelity Investments, administers $7.2 trillion client assets, announced new separatpany called Fidelity Digital A

### Stemming

In [21]:
# ps = PorterStemmer()
# df['stem'] = df['preprocess'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()])) 
# print(df['stem'])

0        cryptoasset enthusiast investor. economics. oct 18 whi bitcoin crypto have No futur whi bitcoin cryptoasset futur nyse creat cryptocurr trade platform bakkt collabor microsoft starbuck physic back bitcoin futur contracts. fidel offer client mine sinc 2015....
1        deduct tith straight worker salaries, archbishop beg govt publish octob 29, 2018 kampala bishop, Dr cyprian kizito lwanga. source: citizentv. the archbishop kampala, cyprian kizito lwanga, suggest govern uganda deduct tith worker salaries. speak mass, cath...
2        tommi robinson could make 1m potenti trip US next month, make one best fund far-right figur sinc second world war, analysi anti-fascist campaign says. the english defenc leagu founder, whose real name stephen yaxley-lennon, invit give address washington me...
3        bitcoin fidel made easier hedg fund pro invest cryptocurr fidel investments, administ $7.2 trillion client assets, announc new separatpani call fidel digit asset servic monday. the fir

### Lemmentization

In [22]:
# # nltk.download('wordnet')
# lmtzr = WordNetLemmatizer()
# df['lem'] = df['preprocess'].apply(lambda x: ' '.join([lmtzr.lemmatize(word, 'v') for word in x.split()])) 
# print(df['lem'])

0        Cryptoasset enthusiast investor. economics. Oct 18 Why Bitcoin Crypto Have No Future Why Bitcoin cryptoassets future NYSE create cryptocurrency trade platform Bakkt collaboration Microsoft Starbucks physically back Bitcoin futures contracts. Fidelity offer...
1        Deduct tithe straight workers salaries, Archbishop beg govt Published October 29, 2018 Kampala Bishop, Dr Cyprian Kizito Lwanga. Source: CitizenTV. The Archbishop Kampala, Cyprian Kizito Lwanga, suggest government Uganda deduct tithe workers salaries. Spea...
2        Tommy Robinson could make 1m potential trip US next month, make one best fund far-right figure since second world war, analysis anti-fascist campaigners says. The English Defence League founder, whose real name Stephen Yaxley-Lennon, invite give address Wa...
3        Bitcoin Fidelity make easier hedge fund pros invest cryptocurrencies Fidelity Investments, administer $7.2 trillion client assets, announce new separatpany call Fidelity Digital Asset 

### Captilization

In [23]:
# df['cap'] = df['preprocess'].apply(lambda x: ' '.join([word.upper() for word in x.split()])) 
# print(df['cap'])

0        CRYPTOASSET ENTHUSIAST INVESTOR. ECONOMICS. OCT 18 WHY BITCOIN CRYPTO HAVE NO FUTURE WHY BITCOIN CRYPTOASSETS FUTURE NYSE CREATING CRYPTOCURRENCY TRADING PLATFORM BAKKT COLLABORATION MICROSOFT STARBUCKS PHYSICALLY BACKED BITCOIN FUTURES CONTRACTS. FIDELITY...
1        DEDUCT TITHES STRAIGHT WORKERS SALARIES, ARCHBISHOP BEGS GOVT PUBLISHED OCTOBER 29, 2018 KAMPALA BISHOP, DR CYPRIAN KIZITO LWANGA. SOURCE: CITIZENTV. THE ARCHBISHOP KAMPALA, CYPRIAN KIZITO LWANGA, SUGGESTED GOVERNMENT UGANDA DEDUCT TITHE WORKERS SALARIES. ...
2        TOMMY ROBINSON COULD MAKE 1M POTENTIAL TRIP US NEXT MONTH, MAKING ONE BEST FUNDED FAR-RIGHT FIGURES SINCE SECOND WORLD WAR, ANALYSIS ANTI-FASCIST CAMPAIGNERS SAYS. THE ENGLISH DEFENCE LEAGUE FOUNDER, WHOSE REAL NAME STEPHEN YAXLEY-LENNON, INVITED GIVE ADDR...
3        BITCOIN FIDELITY MADE EASIER HEDGE FUNDS PROS INVEST CRYPTOCURRENCIES FIDELITY INVESTMENTS, ADMINISTERS $7.2 TRILLION CLIENT ASSETS, ANNOUNCED NEW SEPARATPANY CALLED FIDELITY DIGITAL A

In [24]:
df.to_csv("..\Data_json_csv\preprocess_news_data.csv", index=False)